Hyper Parameter Optimization, aka HPO, is to find a well-performed hyper-parameter on a given search space. The most well-known HPO is grid-search but it only performs well on tiny search space. To resolve hpo on large search space, a lot of algorithms are applied. For example, bayesian optimization is designed for optimizing expensive, black box functions which is very suitable for hpo task. Cost-Frugal optimization on the other hand, taking the training cost into consideration and is aimed to find a better solution within limited cost.

One thing to note is even though hpo is a very activate research field and a lot of algorithms have been invented in the last few years, there's still lacking a general, all-in-one hpo alogrithm that performs well on all datasets. So the best way to find out the right hpo algorithm is always try different hpos on your dataset.

AutoML.Net provides several hpos for you to try out, and you can configure and replace different hpos easily for `AutoMLExperiment` via setting different tuner. In this notebook, we'll go through the following topics.
- Available tuners in AutoML.Net, and how to use it.
- Comparing the performance for those tuners.

In [ ]:
// install dependencies and import using statement
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: Plotly.NET.CSharp, 0.0.1"

// make sure you are using Microsoft.ML.AutoML later than 0.20.0.
#r "nuget: Microsoft.ML.AutoML, 0.20.0-preview.22514.1"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22514.1"
// Import usings.
using System;
using System.IO;
using System.Net;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using Microsoft.ML.SearchSpace;
using Newtonsoft.Json;
using Microsoft.ML.AutoML.CodeGen;
using Microsoft.Data.Analysis;
using Microsoft.ML.SearchSpace.Option;

### Available Tuners in AutoML.Net
For now, those tuners are available in AutoML.Net
- CostFrugalTuner: low-cost HPO algorithm, this is an implementation of [Frugal Optimization for Cost-related Hyperparameters](https://arxiv.org/abs/2005.01571).
- SMAC: Bayesian optimziation using random forest as regression model.
- EciCostFrugalTuner: CostFrugalTuner for hierarchical search space. This will be used as default tuner if `AutoMLExperiment.SetPipeline` get called.
- GridSearch
- RandomSearch

The following section shows how to use different tuner in `AutoMLExperiment`.

In [ ]:
var context = new MLContext(1);
var experiment = context.Auto().CreateExperiment();

// use EciCostFrugalTuner
// Note: EciCostFrugalTuner will be set as default tuner if you call 
// experiment.SetPipeline()
experiment.SetEciCostFrugalTuner();

// use CostFrugalTuner
experiment.SetCostFrugalTuner();

// use SMAC
experiment.SetSmacTuner();

// use GridSearch
experiment.SetGridSearchTuner(step: 10);

// use RandomSearch
experiment.SetRandomSearchTuner(seed: 1);

### Compare GridSearch and EciCostFrugal on titanic dataset

The following section shows how different hpo effect automl performance, by comparing metric trend from GridSearch and EciCostFrugal on titanic dataset.

## Download titanic if necessary

In [ ]:
string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

### Load Dataset

In [ ]:
var trainDataPath = EnsureDataSetDownloaded("titanic-train.csv");
var df = DataFrame.LoadCsv(trainDataPath);

var trainTestSplit = context.Data.TrainTestSplit(df, 0.1);
df.Head(10)

### Construct pipeline and AutoMLExperiment

In [ ]:
var pipeline = context.Auto().Featurizer(df, excludeColumns: new[]{"Survived"})
                        .Append(context.Transforms.Conversion.ConvertType("Survived", "Survived", DataKind.Boolean))
					    .Append(context.Auto().BinaryClassification(labelColumnName: "Survived"));
// Configure AutoML
var monitor = new NotebookMonitor(pipeline);

var experiment = context.Auto().CreateExperiment()
                    .SetPipeline(pipeline)
                    .SetTrainingTimeInSeconds(10)
                    .SetDataset(trainTestSplit.TrainSet, trainTestSplit.TestSet)
                    .SetBinaryClassificationMetric(BinaryClassificationMetric.Accuracy, "Survived", "PredictedLabel")
                    .SetMonitor(monitor);


### Run HPO using GridSearch

In [ ]:
experiment.SetGridSearchTuner(step: 10);
await experiment.RunAsync();
var gridSearchTrial = monitor.CompletedTrials.ToArray();
monitor.CompletedTrials.Clear();

### Run HPO using EciCostFrugal

In [ ]:
experiment.SetEciCostFrugalTuner();
await experiment.RunAsync();
var eciSearchTrials = monitor.CompletedTrials.ToArray();
monitor.CompletedTrials.Clear();

### Compare HPO performace among GridSearch, EciCostFrugal

In [ ]:
using Plotly.NET;

var gridSearchChart = Chart2D.Chart.Line<int, float, string>(gridSearchTrial.Select(t => t.TrialSettings.TrialId), gridSearchTrial.Select(t => (float)t.Metric), Name: "grid_search");
var eciCfoSearchChart = Chart2D.Chart.Line<int, float, string>(eciSearchTrials.Select(t => t.TrialSettings.TrialId), eciSearchTrials.Select(t => (float)t.Metric), Name: "eci_cfo");
var combineChart = Chart.Combine(new[]{ gridSearchChart, eciCfoSearchChart});
combineChart.Display()

## Continue learning

> [⏪ Last Module - AutoML Tuner](./06-AutoML%20HPO%20and%20tuner.ipynb)

## See also

- [AutoML SearchSpace](./Parameter%20and%20SearchSpace.ipynb)